<div style="background-color: #ffffff; color: #000000; padding: 10px;">
<img src="../media/kisz_logo.png" width="192" height="69"> 
<h1> Video Search
<h2> Finding Locations in Audio and Video with Automatic Speech Recognition and Semantic Search
</div>

<div style="background-color: #f6a800; color: #ffffff; padding: 10px;">
    <h2> Part 2 - Byte Pair Encoding
</div>

When working with language models like OpenAI's Whisper or large language models (LLMs), we need a way to represent text in a format that neural networks can understand. Word embeddings provide this bridge by converting discrete language units (e.g., words, punctuation) into continuous vectors of real numbers (e.g., `[-0.342, 0.234, 0.633, 0.45]`). These vectors capture semantic and syntactic information, enabling models to recognise patterns, similarities, and relationships in language data. Without embeddings, models would struggle to process raw text efficiently or generalise across linguistic contexts.

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>1. Tokenisation - Breaking down Text into small Units
</div>

In order to create embeddings, we need to tokenise the text that we will use for model training. Tokenisation is the process of breaking down continuous texts into smaller units such as words and punctuation characters. Each unique token will get its own ID. In the following, tokenisation is illustrated in the context of the novel *Moby Dick*, which can be downloaded from the *Gutenberg Corpus*.

In [3]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

# get text from novel Moby Dick
moby_dick = gutenberg.words('melville-moby_dick.txt')
moby_dick = ' '.join(moby_dick)

# print a few words to get a feeling for the data
print(moby_dick[20000:20100])
print("Total number of characters:", len(moby_dick))

meet a whale - ship on the ocean without being struck by her near appearance . The vessel under shor
Total number of characters: 1259862


[nltk_data] Downloading package gutenberg to /home/hanmul/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


For creating tokens, we first split the text into words and punctuation characters. Each of these *units* will be the basis for a token. These units will not be our final tokens, because some units will be further decomposed, as is detailled out below.

In [4]:
import re

# split the text into tokens
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', moby_dick)

# remove empty tokens
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# investigate
print(preprocessed[:30])

['[', 'Moby', 'Dick', 'by', 'Herman', 'Melville', '1851', ']', 'ETYMOLOGY', '.', '(', 'Supplied', 'by', 'a', 'Late', 'Consumptive', 'Usher', 'to', 'a', 'Grammar', 'School', ')', 'The', 'pale', 'Usher', '--', 'threadbare', 'in', 'coat', ',']


Next, we convert the text tokens into token IDs, representing our vocabulary. The vocabulary consists of each unique token, sorted alphabetically. Each unique token is then mapped onto its ID.

In [6]:
# create the vocabulary
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f'vocabulary size: {vocab_size} unique tokens.')

# print the first items of the vocabulary
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 10:
        break

vocabulary size: 19243 unique tokens.
('!', 0)
('"', 1)
('$', 2)
('&', 3)
("'", 4)
('(', 5)
(')', 6)
('*', 7)
(',', 8)
('-', 9)
('--', 10)


<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>2. Special Context Tokens
</div>

Often, special tokens are added to the vocabulary for different reasons.
- `<|UNK|>` (unknown word) denotes out-of-vocabulary words
- `<|endoftext|>` or `<|EOS|>` (end of sequence) are used, among others, when multiple texts such as newspaper articles are concatenated
- `<|beginningoftext|>` or `<|BOS|>` denot beginnings analogous to `<|endoftext|>` and `<|EOS|>`
- `<|PAD|>` (padding) is used when training LLMs with batch sizes greater than 1 (i.e., inputs with different length are 'padded' to the length of the longest input; e.g., `['Never', 'mind', '<|PAD|>', '<|PAD|>', '<|PAD|>']` has the same length as `['Actions', 'speak', 'louder', 'than', 'words']`

For illustrating the concept of special tokens, let us look at an example, where we encounter an unknown word, that is, a word that is not included in our vocabulary. Encoding this text does not work, because the word has no ID associated with it. In the folowing example, the out-of-vocabulary word is "Hello".


In [11]:
# add <|unk|> to vocabulary
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|UNK|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

# check whether it worked
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('zone', 19239)
('zoned', 19240)
('zones', 19241)
('zoology', 19242)
('<|UNK|>', 19243)


We can now define `encode` and `decode` functions for a simple tokeniser. In the AI domain, encoding refers to the process of transforming a input sequence into tokens, whereas decoding refers to the reverse transformation, that is, tokens to output sequence. In the case of this notebook, input and output sequences are texts. With the `encode` and the `decode` function, we can turn text into IDs and vice versa.

In [17]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|UNK|>" for item in preprocessed # add <|UNK|> for unknown tokens
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [18]:
sentence = sorted(set(['this', 'sentence', 'is', 'a', 'test', 'sentence', '.']))
vocab_small = {token:integer for integer,token in enumerate(sentence)}

tokenizer = SimpleTokenizer(vocab_small)

# print test sentence and corresponding token IDs
ids = tokenizer.encode('this sentence is a test sentence.')
print(tokenizer.decode(ids))
print(ids)

this sentence is a test sentence.
[5, 3, 2, 1, 4, 3, 0]


After sorting alphabetically, the "." is the first token, which gets the ID 0. The second token (ID 1) is "a". The token "sentence", which occurs twice, get the ID 3. Using the toy vocabulary, the relationship between tokens and IDs is straightforward. However, in real texts that are much longer, it is not possible anymore to see this relationship at one glance.

In [19]:
tokenizer = SimpleTokenizer(vocab)

text = moby_dick[20015:20200]

# eprint text and corresponding IDs
ids = tokenizer.encode(text)
print(tokenizer.decode(ids))
print(ids)

ship on the ocean without being struck by her near appearance. The vessel under short sail, with look - outs at the mast - heads, eagerly scanning the wide expanse around them, has
[15641, 12867, 17275, 12802, 19050, 4821, 16694, 5471, 9992, 12548, 4227, 11, 3296, 18487, 17981, 15683, 15152, 8, 19037, 11657, 9, 12993, 4435, 17275, 11960, 9, 9897, 8, 7843, 15255, 17275, 18965, 8389, 4322, 17280, 8, 9839]


Out-of-vocabulary words are encoded with the ID that represents the special token `<|UNK|>`.

In [20]:
tokenizer = SimpleTokenizer(vocab)

text = "Hello World"

print(tokenizer.decode(tokenizer.encode(text)))
print(tokenizer.encode(text))

<|UNK|> World
[19243, 3640]


<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>3. Byte Pair Encoding
</div>

TO-DO
- Explain motivation
- long-tailed word distribution
- repetitive patterns in longer words
- relationship vocabulary size and training
- examples vocabulary size different LLMs

Whisper uses Byte Pair encoding (BPE) as its tokeniser. BPE breaks down words into smaller subword units or even individual characters in order to handle out-of-vocabulary words instead of using an `<|UNK|>` token. For instance, an out-of-vocabulary word such as "Uncharacteristically" may be broken down into the tokens `['Unchar', 'acter', 'isti', 'cally']` or some other subword breakdown, depending on how BPE is implemented.

The original BPE tokeniser that OpenAI implemented for training the original GPT models can be found [here](https://github.com/openai/gpt-2/blob/master/src/encoder.py). The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage. Most projects use OpenAI's open-source [tiktoken library](https://github.com/openai/tiktoken) due to its computational performance; it allows loading pretrained GPT-2 and GPT-4 tokenisers, for example (the Llama 3 models were trained using the GPT-4 tokeniser). There's also an implementation called [minBPE](https://github.com/karpathy/minbpe) with training support. Additionally, Hugging Face tokenisers are also capable of training and loading various tokenisers; see [this GitHub discussion](https://github.com/rasbt/LLMs-from-scratch/discussions/485) by a reader who trained a BPE tokenizer on the Nepali language for more info.

## The Main Idea behind Byte Pair Encoding (BPE)

## Bits and bytes

- Before getting to the BPE algorithm, let's introduce the notion of bytes
- Consider converting text into a byte array (BPE stands for "byte" pair encoding after all):

In [21]:
text = "Hello World"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'Hello World')


- When we call `list()` on a `bytearray` object, each byte is treated as an individual element, and the result is a list of integers corresponding to the byte values:

In [22]:
ids = list(byte_ary)
print(ids)

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]


- This would be a valid way to convert text into a token ID representation that we need for the embedding layer of an LLM
- However, the downside of this approach is that it is creating one ID for each character (that's a lot of IDs for a short text!)
- I.e., this means for a 11-character input text, we have to use 11 token IDs as input to the LLM:

In [23]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 11
Number of token IDs: 11


In [24]:
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("Hello World")

[15496, 2159]

- If you have worked with LLMs before, you may know that the BPE tokenizers have a vocabulary where we have a token ID for whole words or subwords instead of each character
- For example, the GPT-2 tokenizer tokenizes the same text ("Hello World") into only 2 instead of 11 tokens.
- You can double-check this using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library](https://github.com/openai/tiktoken)

Since a byte consists of 8 bits, there are 2<sup>8</sup> = 256 possible values that a single byte can represent, ranging from 0 to 255. You can confirm this by executing the code `bytearray(range(0, 257))`, which will warn you that `ValueError: byte must be in range(0, 256)`). A BPE tokenizer usually uses these 256 values as its first 256 single-character tokens; one could visually check this by running the code below. Note that entries 256 and 257 are not single-character values but double-character values (a whitespace + a letter).

In [26]:
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(250, 260):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

250: �
251: �
252: �
253: �
254: �
255: �
256:  t
257:  a
258: he
259: in


## Building the vocabulary

The goal of the BPE tokenization algorithm is to build a vocabulary of commonly occurring subwords like `298: ent` (which can be found in *entangle, entertain, enter, entrance, entity, ...*, for example):

In [27]:
gpt2_tokenizer.decode([298])

'ent'

... or even complete words like 

In [28]:
for i in [318, 617, 1212, 2420]:
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

318:  is
617:  some
1212: This
2420:  text


The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage. Before we get to the actual code implementation, the form that is used for LLM tokenizers today can be summarized as described in the following sections.

## BPE algorithm outline

**1. Identify frequent pairs**
- In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters)

**2. Replace and record**

- Replace that pair with a new placeholder ID (one not already in use, e.g., if we start with 0...255, the first placeholder would be 256)
- Record this mapping in a lookup table
- The size of the lookup table is a hyperparameter, also called "vocabulary size" (for GPT-2, that's
50,257)

**3. Repeat until no gains**

- Keep repeating steps 1 and 2, continually merging the most frequent pairs
- Stop when no further compression is possible (e.g., no pair occurs more than once)

**Decompression (decoding)**

- To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table



## BPE algorithm example

### Concrete example of the encoding part (steps 1 & 2)

Suppose we have the text (training dataset) `the cat in the hat` from which we want to build the vocabulary for a BPE tokenizer

**Iteration 1**

1. Identify frequent pairs
  - In this text, "th" appears twice (at the beginning and before the second "e")

2. Replace and record
  - replace "th" with a new token ID that is not already in use, e.g., 256
  - the new text is: `<256>e cat in <256>e hat`
  - the new vocabulary is

```
  0: ...
  ...
  256: "th"
```

**Iteration 2**

1. **Identify frequent pairs**  
   - In the text `<256>e cat in <256>e hat`, the pair `<256>e` appears twice

2. **Replace and record**  
   - replace `<256>e` with a new token ID that is not already in use, for example, `257`.  
   - The new text is:
     ```
     <257> cat in <257> hat
     ```
   - The updated vocabulary is:
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**Iteration 3**

1. **Identify frequent pairs**  
   - In the text `<257> cat in <257> hat`, the pair `<257> ` appears twice (once at the beginning and once before “hat”).

2. **Replace and record**  
   - replace `<257> ` with a new token ID that is not already in use, for example, `258`.  
   - the new text is:
     ```
     <258>cat in <258>hat
     ```
   - The updated vocabulary is:
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- and so forth

### Concrete example of the decoding part (step 3)

- To restore the original text, we reverse the process by substituting each token ID with its corresponding pair in the reverse order they were introduced
- Start with the final compressed text: `<258>cat in <258>hat`
-  Substitute `<258>` → `<257> `: `<257> cat in <257> hat`  
- Substitute `<257>` → `<256>e`: `<256>e cat in <256>e hat`
- Substitute `<256>` → "th": `the cat in the hat`

## 2. A simple BPE implementation

Below is an implementation of this algorithm described above as a Python class that mimics the `tiktoken` Python user interface. Note that the encoding part above describes the original training step via `train()`; however, the `encode()` method works similarly (although it looks a bit more complicated because of the special token handling):

1. Split the input text into individual bytes
2. Repeatedly find & replace (merge) adjacent tokens (pairs) when they match any pair in the learned BPE merges (from highest to lowest "rank," i.e., in the order they were learned)
3. Continue merging until no more merges can be applied
4. The final list of token IDs is the encoded output

In [34]:
from collections import Counter, deque
from functools import lru_cache


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # For the official OpenAI GPT-2 merges, use a rank dict:
        # of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with "Ġ"
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        
        processed_text = text
        '''
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)
        '''

        # Initialize vocab with unique characters, including "Ġ" if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )

        '''
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")
        '''

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id



    '''
    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pre-trained vocabulary and BPE merges from OpenAI's GPT-2 files.

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json').
            bpe_merges_path (str): Path to the bpe_merges file  (GPT-2 calls it 'vocab.bpe').
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # Convert loaded vocabulary to correct format
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use an existing token ID as a placeholder for '\n'
            # Preferentially use "<|endoftext|>" if available
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # Load GPT-2 merges and store them with an assigned "rank"
        self.bpe_ranks = {}  # reset ranks
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # If token1 or token2 not in vocab, skip
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} as one token is not in the vocabulary.")
    '''

    def encode(self, text, allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
        import re
    
        token_ids = []
    
        # If special token handling is enabled
        if allowed_special is not None and len(allowed_special) > 0:
            # Build regex to match allowed special tokens
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )
    
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # Encode prefix without special handling
    
                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
    
            text = text[last_index:]  # Remaining part to process normally
    
            # Check if any disallowed special tokens are in the remainder
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")
    
        # If no special tokens, or remaining text after special token split:
        tokens = []
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:
                    tokens.append(" " + word) ############################################################
                elif j == 0:
                    tokens.append(word)
                else:
                    tokens.append(" " + word) ############################################################
    
        for token in tokens:
            if token in self.inverse_vocab:
                token_ids.append(self.inverse_vocab[token])
            else:
                token_ids.extend(self.tokenize_with_bpe(token))
    
        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    i += 2  # Skip the next token as it's merged
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens
        return token_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # Add space if not present before a newline
                decoded_string += token

            #elif token.startswith("Ġ"):
            #    decoded_string += " " + token[1:]

            else:
                decoded_string += token
        return decoded_string

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

## BPE implementation walkthrough

### Training, encoding, and decoding

First, let's consider some sample text as our training dataset:

- Next, let's initialize and train the BPE tokenizer with a vocabulary size of 1,000
- Note that the vocabulary size is already 256 by default due to the byte values discussed earlier, so we are only "learning" 744 vocabulary entries (if we consider the `<|endoftext|>` special token and the `Ġ` whitespace token; so, that's 742 to be precise)
- For comparison, the GPT-2 vocabulary is 50,257 tokens, the GPT-4 vocabulary is 100,256 tokens (`cl100k_base` in tiktoken), and GPT-4o uses 199,997 tokens (`o200k_base` in tiktoken); they have all much bigger training sets compared to our simple example text above

In [31]:
tokenizer = BPETokenizerSimple()
tokenizer.train(moby_dick, vocab_size=1000, allowed_special={"<|endoftext|>"})

In [33]:
# investigate length of vocab (should be as long as our parameter for vocab_size, i.e., 1000)
print(len(tokenizer.vocab))

# investigate last five items in vocab
print(list(tokenizer.vocab.items())[-5:])

1000
[(995, '," '), (996, 'bra'), (997, 'ct'), (998, 'ive '), (999, 'see ')]


This vocabulary is created by merging 743 times (`= 1000 - len(range(0, 256)) - len(special_tokens) = 1000 - 256 - 1 = 743`). This means that the first 256 entries are single-character tokens. Next, let's use the created merges via the `encode` method to encode some text:

In [59]:
input_text = "A person who never made a mistake never tried anything new."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[65, 32, 411, 677, 110, 32, 295, 111, 32, 900, 404, 32, 479, 464, 32, 97, 32, 836, 289, 578, 101, 32, 900, 404, 258, 345, 336, 32, 266, 121, 277, 272, 32, 900, 119, 46]


In [60]:
input_text = "A person who never made a mistake never tried anything new.<|endoftext|> "
token_ids = tokenizer.encode(input_text)
print(token_ids)

[65, 32, 411, 677, 110, 32, 295, 111, 32, 900, 404, 32, 479, 464, 32, 97, 32, 836, 289, 578, 101, 32, 900, 404, 258, 345, 336, 32, 266, 121, 277, 272, 32, 900, 119, 46, 60, 124, 270, 641, 102, 116, 441, 116, 124, 62]


In [61]:
input_text = "A person who never made a mistake never tried anything new.<|endoftext|> "
token_ids = tokenizer.encode(input_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[65, 32, 411, 677, 110, 32, 295, 111, 32, 900, 404, 32, 479, 464, 32, 97, 32, 836, 289, 578, 101, 32, 900, 404, 258, 345, 336, 32, 266, 121, 277, 272, 32, 900, 119, 46, 256]


In [62]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 73
Number of token IDs: 37


From the lengths above, we can see that a 73-character sentence was encoded into 37 token IDs, effectively cutting the input length roughly in half compared to a character-byte-based encoding. Note that the vocabulary itself is used in the `decode()` method, which allows us to map the token IDs back into text:

In [65]:
print(tokenizer.decode(token_ids))

A person who never made a mistake never tried anything new.<|endoftext|>


Iterating over each token ID can give us a better understanding of how the token IDs are decoded via the vocabulary:

In [66]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

65 -> A
32 ->  
411 -> per
677 -> so
110 -> n
32 ->  
295 -> wh
111 -> o
32 ->  
900 -> ne
404 -> ver
32 ->  
479 -> ma
464 -> de
32 ->  
97 -> a
32 ->  
836 -> mi
289 -> st
578 -> ak
101 -> e
32 ->  
900 -> ne
404 -> ver
258 ->  t
345 -> ri
336 -> ed
32 ->  
266 -> an
121 -> y
277 -> th
272 -> ing
32 ->  
900 -> ne
119 -> w
46 -> .
256 -> <|endoftext|>


As we can see, most token IDs represent 2-character subwords; that's because the training data text is very short with not that many repetitive words, and because we used a relatively small vocabulary size. As a summary, calling `decode(encode())` should be able to reproduce arbitrary input texts:

In [67]:
tokenizer.decode(
    tokenizer.encode("This is some text.")
)

'This is some text.'

In [68]:
tokenizer.decode(
    tokenizer.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

## OpenAI's open-source tiktoken library

In practice, I highly recommend using [tiktoken](https://github.com/openai/tiktoken) because above implementation focuses on readability and educational purposes but not on performance. The BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library implements its core algorithms in Rust to improve computational performance.

In [ ]:
import importlib

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [70]:
tokenizer = tiktoken.get_encoding("gpt2")

In [75]:
text = "A Byte Pair Encoding (BPE) tokenizer is a subword tokenization algorithm that iteratively " \
"merges the most frequent pairs of characters or character sequences in a text to build a vocabulary of " \
"common subword units, enabling efficient and flexible representation of words."

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[32, 30589, 39645, 14711, 7656, 357, 33, 11401, 8, 11241, 7509, 318, 257, 850, 4775, 11241, 1634, 11862, 326, 11629, 9404, 4017, 3212, 262, 749, 10792, 14729, 286, 3435, 393, 2095, 16311, 287, 257, 2420, 284, 1382, 257, 25818, 286, 2219, 850, 4775, 4991, 11, 15882, 6942, 290, 12846, 10552, 286, 2456, 13]


In [76]:
strings = tokenizer.decode(integers)

print(strings)

A Byte Pair Encoding (BPE) tokenizer is a subword tokenization algorithm that iteratively merges the most frequent pairs of characters or character sequences in a text to build a vocabulary of common subword units, enabling efficient and flexible representation of words.


## 2.6 Data sampling with a sliding window

- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- For each text chunk, we want the inputs and targets
- Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, the prediction would look like as follows:

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
- For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one

- Install and import PyTorch (see Appendix A for installation tips)

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- We use a sliding window approach, changing the position by +1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Create dataset and dataloader that extract chunks from the input text dataset

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- An example using stride equal to the context length (here: 4) as shown below:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- We can also create batched outputs
- Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

- The data is already almost ready for an LLM
- But lastly let us embed the tokens in a continuous vector representation using an embedding layer
- Usually, these embedding layers are part of the LLM itself and are updated (trained) during model training

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Suppose we have the following four input examples with input ids 2, 3, 5, and 1 (after tokenization):

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want to create embeddings of size 3:

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- This would result in a 6x3 weight matrix:

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully-connected layer, which is described in the supplementary code in [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)
- Because the embedding layer is just a more efficient implementation that is equivalent to the one-hot encoding and matrix-multiplication approach it can be seen as a neural network layer that can be optimized via backpropagation

- To convert a token with id 3 into a 3-dimensional vector, we do the following:

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Note that the above is the 4th row in the `embedding_layer` weight matrix
- To embed all four `input_ids` values above, we do

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- An embedding layer is essentially a look-up operation:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 Encoding word positions

- Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- The BytePair encoder has a vocabulary size of 50,257:
- Suppose we want to encode the input tokens into a 256-dimensional vector representation:

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
- If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


- In the initial phase of the input processing workflow, the input text is segmented into separate tokens
- Following this segmentation, these tokens are transformed into token IDs based on a predefined vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">